In [ ]:
!nvidia-smi

In [3]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [4]:
!pip install ultralytics==8.0.196

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.3/78.2 GB disk)


In [5]:
from ultralytics import YOLO
from IPython.display import display, Image

In [6]:
!yolo task=detect mode=train model=yolov8s.pt data={"/content/drive/MyDrive/supermarket_dataset"}/data.yaml epochs=25 imgsz=640 plots=True

100% 21.5M/21.5M [00:00<00:00, 164MB/s]
New https://pypi.org/project/ultralytics/8.1.10 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/drive/MyDrive/supermarket_dataset/data.yaml, epochs=25, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_

In [7]:
model = YOLO(f'/content/runs/detect/train/weights/best.pt')
results = model(source='/content/cleaning-store.jpg', conf=0.5)


image 1/1 /content/cleaning-store.jpg: 384x640 2 100- O-O-Ss, 97.8ms
Speed: 3.2ms preprocess, 97.8ms inference, 487.7ms postprocess per image at shape (1, 3, 384, 640)


In [13]:
from ultralytics import YOLO
from PIL import Image

# Load the model
model = YOLO('/content/runs/detect/train/weights/best.pt')  # Adjust path as necessary

# Perform detection
results = model(source='/content/coke.jpg', conf=0.25)

# Show the results
for r in results:
    im_array = r.plot()  # plot a BGR numpy array of predictions
    im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image
    im.show()  # show image
    im.save('results.jpg')  # save image


image 1/1 /content/coke.jpg: 416x640 6 100- O-O-Ss, 13.5ms
Speed: 1.8ms preprocess, 13.5ms inference, 1.9ms postprocess per image at shape (1, 3, 416, 640)


In [26]:
from ultralytics import YOLO
import torch


model = YOLO('/content/runs/detect/train/weights/best.pt')  # Adjust path as necessary

# Perform detection
results = model(source='/content/coke.jpg', conf=0.25)

boxes = []
# View results
for r in results:
    boxes.append(r.boxes.xyxy.tolist())


print(boxes)


image 1/1 /content/coke.jpg: 416x640 6 100- O-O-Ss, 13.5ms
Speed: 2.1ms preprocess, 13.5ms inference, 1.9ms postprocess per image at shape (1, 3, 416, 640)


[[[210.02906799316406, 33.51850891113281, 248.0020294189453, 72.35305786132812], [392.38165283203125, 33.973846435546875, 433.01580810546875, 81.1351547241211], [109.13330078125, 139.58413696289062, 189.38609313964844, 209.69168090820312], [183.63577270507812, 211.16404724121094, 342.47735595703125, 279.0], [221.88134765625, 141.2664337158203, 274.8403015136719, 209.2200927734375], [223.22132873535156, 141.098388671875, 315.2217102050781, 209.4232177734375]]]


In [24]:
!pip install singlestoredb


In [29]:
import singlestoredb as s2
conn = s2.connect('ayush:Test1234@svc-3482219c-a389-4079-b18b-d50662524e8a-shared-dml.aws-virginia-6.svc.singlestore.com:3333/database_79fb0')


In [35]:
# SQL command to create the BoundingBoxes table
create_table_command = """
CREATE TABLE IF NOT EXISTS BoundingBoxes (
    BoxID INT AUTO_INCREMENT PRIMARY KEY,
    ImageID INT,
    X1 FLOAT,
    Y1 FLOAT,
    X2 FLOAT,
    Y2 FLOAT
);
"""



# Execute the SQL command
with s2.connect('ayush:Test1234@svc-3482219c-a389-4079-b18b-d50662524e8a-shared-dml.aws-virginia-6.svc.singlestore.com:3333/database_79fb0') as conn:
    with conn.cursor() as cursor:
        cursor.execute(create_table_command)
        print("Table 'BoundingBoxes' created successfully.")

Table 'BoundingBoxes' created successfully.


In [37]:
# Prepare the SQL statement for inserting bounding box data
stmt = 'INSERT INTO BoundingBoxes (ImageID, X1, Y1, X2, Y2) VALUES (%s, %s, %s, %s, %s)'

# Assuming each bounding box is associated with the same image, for example, ImageID = 1
image_id = 1  # Adjust this based on your actual ImageID

with s2.connect('ayush:Test1234@svc-3482219c-a389-4079-b18b-d50662524e8a-shared-dml.aws-virginia-6.svc.singlestore.com:3333/database_79fb0') as conn:
    conn.autocommit(True)
    with conn.cursor() as cur:
        # Iterate over each bounding box and insert its data
        for box in boxes[0]:  # Assuming boxes[0] contains your bounding box data for one image
            cur.execute(stmt, (image_id, box[0], box[1], box[2], box[3]))
